In [276]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, ConvLSTM2D, Dropout, Flatten, RepeatVector, Reshape, TimeDistributed, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping


In [277]:
segments = 15
# Set the time step in minutes
time_step = 15
# Set the start and end times for the time window

start_time = '06:00:00'

end_time = '19:00:00'


In [278]:
initial = pd.read_csv("data/bus_running_times_feature_added_all.csv")

In [279]:
initial.head()

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds,length,...,rt(n-2),rt(n-3),hour_of_day,day,month,temp,precip,windspeed,conditions,dt(n-1)
0,1.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0,0.6261,...,96.0,96.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0,1.2808,...,247.0,247.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,74.0
2,1.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0,2.1125,...,69.0,506.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0
3,1.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0,1.5513,...,210.0,69.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,6.0
4,1.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0,0.8450,...,496.0,210.0,6.0,1.0,10.0,20.0,0.0,6.1,Partially cloudy,0.0


In [280]:
busses_new = initial[['trip_id','deviceid','direction','segment','date','start_time','end_time','run_time','run_time_in_seconds']]
busses_new.head()

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds
0,1.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0
1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0
2,1.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0
3,1.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0
4,1.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0


In [281]:
busses_new.dropna(inplace=True)

C:\Users\chath\AppData\Local\Temp\ipykernel_72\50537125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  busses_new.dropna(inplace=True)


In [282]:
busses_new = busses_new[busses_new['segment'] <= segments]

In [283]:
len(busses_new)

103843

In [284]:
num_rows_with_nan = busses_new.isna().any(axis=1).sum()
print(f'Number of rows with NaN values: {num_rows_with_nan}')

Number of rows with NaN values: 0


In [285]:
# Convert the 'date' column to a datetime object
busses_new['date'] = pd.to_datetime(busses_new['date'])

# Create a boolean mask to filter rows with dates on or after 10/1/2022
mask = busses_new['date'] >= '2022-10-01'

# Create the train and test dataframes
busses_train = busses_new[~mask]
busses_test = busses_new[mask]

In [286]:
def getAvgRuntimeWithSegment(dataframe,time_step,num_segments):
    
    dataframe = dataframe.copy()

    # Convert the 'date' and 'start_time' columns to datetime objects
    dataframe['start_time'] = pd.to_datetime(busses_train['start_time'])
    # Set the start and end times for the time window
    start_time = pd.to_datetime('06:00:00').time()
    end_time = pd.to_datetime('19:00:00').time()

    # Create a list to store the data for the new dataframe
    data = []
    
    # Iterate over the unique dates in the 'date' column
    for date in dataframe['date'].unique():
        # Filter the rows for the current date
        df_date = dataframe[dataframe['date'] == date]
        df_date.loc[:, 'start_time'] = df_date['start_time'].dt.time  # Use .loc to modify the original DataFrame
    
        # Create a time range for the current date with the specified time step
        time_range = pd.date_range(date, date + pd.Timedelta(days=1), freq=f'{time_step}T')
        
        # Iterate over the time range
        for start, end in zip(time_range[:-1], time_range[1:]):
            # Check if the start time is within the specified time window
            if start.time() >= start_time and start.time() < end_time:
                # Filter the rows for the current time window
                mask = (df_date['start_time'] >= start.time()) & (df_date['start_time'] < end.time())
                df_time_window = df_date[mask]
                # Calculate the average run time for each segment in the current time window
                avg_run_time = df_time_window.groupby('segment')['run_time_in_seconds'].mean().reset_index()
    
                # Append the data to the data list
                for row in avg_run_time.itertuples():
                    data.append((date, start.time(), row.segment, row.run_time_in_seconds))
    
    
    # Create a new dataframe from the data list
    df_avg_run_time = pd.DataFrame(data, columns=['date', 'start_time', 'segment', 'avg_run_time'])
    # Create a MultiIndex with all combinations of date, start_time, and segment
    idx = pd.MultiIndex.from_product([df_avg_run_time['date'].unique(), df_avg_run_time['start_time'].unique(), range(1, num_segments + 1)], names=['date', 'start_time', 'segment'])
    
    # Reindex the dataframe using the new MultiIndex
    df_avg_run_time = df_avg_run_time.set_index(['date', 'start_time', 'segment']).reindex(idx, fill_value=0).reset_index()
    return df_avg_run_time


In [287]:
df = getAvgRuntimeWithSegment(busses_train,time_step,segments)
df.head()

C:\Users\chath\AppData\Local\Temp\ipykernel_72\591756120.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe['start_time'] = pd.to_datetime(busses_train['start_time'])


,date,start_time,segment,avg_run_time
0,2021-10-01,06:30:00,1,69.0
1,2021-10-01,06:30:00,2,210.0
2,2021-10-01,06:30:00,3,0.0
3,2021-10-01,06:30:00,4,0.0
4,2021-10-01,06:30:00,5,0.0


In [288]:
df.head(50)

,date,start_time,segment,avg_run_time
0,2021-10-01,06:30:00,1,69.0
1,2021-10-01,06:30:00,2,210.0
2,2021-10-01,06:30:00,3,0.0
3,2021-10-01,06:30:00,4,0.0
4,2021-10-01,06:30:00,5,0.0
5,2021-10-01,06:30:00,6,0.0
6,2021-10-01,06:30:00,7,0.0
7,2021-10-01,06:30:00,8,0.0
8,2021-10-01,06:30:00,9,0.0
9,2021-10-01,06:30:00,10,0.0


In [289]:
def fillTimeSteps(df_avg_run_time, startTime, endTime, time_step, num_segments):
    start_Time = pd.to_datetime(startTime).time()

    end_Time = pd.to_datetime(endTime).time()
    # Create a new DataFrame to store the missing data
    data = []

    # Iterate over the unique dates in the 'date' column
    for date in df_avg_run_time['date'].unique():
        # Filter the rows for the current date
        df_date = df_avg_run_time[df_avg_run_time['date'] == date]

        # Create a time range for the current date with the specified time step
        time_range = pd.date_range(date, date + pd.Timedelta(days=1), freq=f'{time_step}T').time

        # Iterate over the time range
        for t in time_range:
            # Check if the start time is within the specified time window
            if start_Time <= t < end_Time:
                # Check if the current start time is present in the dataframe
                if not (df_date['start_time'] == t).any():
                    # Add rows for the missing start time and segments
                    for segment in range(1, num_segments + 1):
                        data.append((date, t, segment, 0))

    # Create a new DataFrame from the data list
    df_missing = pd.DataFrame(data, columns=['date', 'start_time', 'segment', 'avg_run_time'])

    # Concatenate the new DataFrame with the original DataFrame
    df_avg_run_time = pd.concat([df_avg_run_time, df_missing], ignore_index=True)
    # Sort the rows of the dataframe by the 'date', 'start_time', and 'segment' columns
    df_avg_run_time = df_avg_run_time.sort_values(by=['date', 'start_time', 'segment'])
    return df_avg_run_time

In [290]:
df2 = fillTimeSteps(df,start_time,end_time,time_step,segments)

In [291]:
df2.head(50)

,date,start_time,segment,avg_run_time
765,2021-10-01,06:00:00,1,0.0
766,2021-10-01,06:00:00,2,0.0
767,2021-10-01,06:00:00,3,0.0
768,2021-10-01,06:00:00,4,0.0
769,2021-10-01,06:00:00,5,0.0
770,2021-10-01,06:00:00,6,0.0
771,2021-10-01,06:00:00,7,0.0
772,2021-10-01,06:00:00,8,0.0
773,2021-10-01,06:00:00,9,0.0
774,2021-10-01,06:00:00,10,0.0


In [292]:
# Count the number of rows with a value of 0 in avg_run_time
num_zero_values = (df2['avg_run_time'] == 0).sum()

print(f"Number of rows with 0 avg_run_time: {num_zero_values}")

Number of rows with 0 avg_run_time: 114914


In [293]:
# To handle outliers in the avg_run_time column of your DataFrame using the absolute deviation around the median method, you can follow these steps:
# 
#     Calculate the median of the avg_run_time column.
#     Calculate the absolute deviations of each value from the median.
#     Calculate the median absolute deviation (MAD) using the absolute deviations.
#     Set a threshold (for example, a multiple of MAD) to identify outliers.
#     Replace the outlier values with a suitable value (e.g., the median).

def handleOutliers(df):
    # Calculate the median of avg_run_time
    median_avg_run_time = df['avg_run_time'].median()
    
    # Calculate the absolute deviations
    df['abs_deviation'] = np.abs(df['avg_run_time'] - median_avg_run_time)
    
    # Calculate the median absolute deviation (MAD)
    mad = df['abs_deviation'].median()
    
    # Set a threshold (for example, 3 times MAD) to identify outliers
    threshold = 3 * mad
    
    # Replace outliers with the median value
    df.loc[df['abs_deviation'] > threshold, 'avg_run_time'] = median_avg_run_time
    
    # Drop the temporary column
    df = df.drop(columns=['abs_deviation'])
    
    return df

In [294]:
df3 = handleOutliers(df2)

In [295]:
df3.head(150)

,date,start_time,segment,avg_run_time
765,2021-10-01,06:00:00,1,0.0
766,2021-10-01,06:00:00,2,0.0
767,2021-10-01,06:00:00,3,0.0
768,2021-10-01,06:00:00,4,0.0
769,2021-10-01,06:00:00,5,0.0
...,...,...,...,...
685,2021-10-01,08:15:00,11,0.0
686,2021-10-01,08:15:00,12,0.0
687,2021-10-01,08:15:00,13,0.0
688,2021-10-01,08:15:00,14,0.0


In [296]:
len(df3)

187200

In [326]:
# Count the number of rows with a value of 0 in avg_run_time
num_zero_values = (df3['avg_run_time'] == 0).sum()

print(f"Number of rows with 0 avg_run_time: {num_zero_values}")

Number of rows with 0 avg_run_time: 187200


In [305]:
df3

,date,start_time,segment,avg_run_time
765,2021-10-01,06:00:00,1,0.0
766,2021-10-01,06:00:00,2,0.0
767,2021-10-01,06:00:00,3,0.0
768,2021-10-01,06:00:00,4,0.0
769,2021-10-01,06:00:00,5,0.0
...,...,...,...,...
187015,2022-09-30,18:45:00,11,0.0
187016,2022-09-30,18:45:00,12,0.0
187017,2022-09-30,18:45:00,13,0.0
187018,2022-09-30,18:45:00,14,0.0


In [321]:
def normalizeSTD(df):
    # Filter out zero values when calculating mean for each segment, start time, and weekday
    mean_data = df.groupby(['segment', 'start_time', df['date'].dt.weekday])['avg_run_time'].mean()
    # Filter out zero values when calculating standard deviation for each segment
    std_data = df.groupby('segment')['avg_run_time'].std()

    print(mean_data)
    print(std_data)
    
    # Normalize values and create a new DataFrame
    normalized_data = []
    for index, row in df.iterrows():
        segment, start_time, weekday = row['segment'], row['start_time'], row['date'].weekday()
        mean_value = mean_data.get((segment, start_time, weekday), 0)  # Default to 0 if not found
        std_value = std_data.get(segment, 1)  # Default to 1 if not found to avoid division by zero
        
        # If the value is non-zero, calculate normalized value; otherwise, keep it as zero
        normalized_value = (row['avg_run_time'] - mean_value) / std_value if row['avg_run_time'] != 0 else 0
        normalized_data.append((row['date'], row['start_time'], segment, normalized_value))
    
    normalized_columns = ['date', 'start_time', 'segment', 'normalized_avg_run_time']
    df_normalized = pd.DataFrame(normalized_data, columns=normalized_columns)
    return df_normalized



In [303]:
def normalzeMinMax(df):
    
    # Calculate the minimum avg_run_time for each segment, start time, and weekday
    min_data = df.groupby(['segment', 'start_time', df['date'].dt.weekday])['avg_run_time'].min()
    max_data = df.groupby(['segment', 'start_time', df['date'].dt.weekday])['avg_run_time'].max()
    
    # Normalize values and create a new DataFrame
    normalized_data = []
    for index, row in df.iterrows():
        segment, start_time, weekday = row['segment'], row['start_time'], row['date'].weekday()
        min_value = min_data.get((segment, start_time, weekday), 0)  # Default to 0 if not found
        max_value = max_data.get((segment, start_time, weekday), 0)  # Default to 0 if not found
        
        if max_value == 0:
            normalized_value = 0
        else:
            normalized_value = (row['avg_run_time'] - min_value) / (max_value - min_value)
        
        normalized_data.append((row['date'], row['start_time'], segment, normalized_value))
    
    normalized_columns = ['date', 'start_time', 'segment', 'normalized_avg_run_time']
    df_normalized = pd.DataFrame(normalized_data, columns=normalized_columns)
    return  df_normalized

In [325]:
df4 = df3[df3['avg_run_time'] != 0]
len(df4)

0